In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
#Cargamos los CSV
reclamos_df = spark.read.option("header", True).csv("../datasets/reclamos_clientes.csv")
clientes_df = spark.read.option("header", True).csv("../datasets/clientes.csv")
atenciones_df = spark.read.option("header", True).csv("../datasets/atenciones.csv")

In [ ]:
reclamos_df.show(5)

In [ ]:
from pyspark.sql.functions import trim, col, lower

In [ ]:
reclamos_df = reclamos_df.dropDuplicates().dropna()
clientes_df = clientes_df.dropDuplicates().dropna()
atenciones_df = atenciones_df.dropDuplicates().dropna()

In [ ]:
reclamos_df = reclamos_df.withColumn("descripcion", lower(trim(col("descripcion"))))
clientes_df = clientes_df.withColumn("nombre", trim(col("nombre"))) \
                         .withColumn("ciudad", trim(col("ciudad")))

In [ ]:
from pyspark.sql.functions import col, count, avg, round,desc

r = reclamos_df.alias("r")
c = clientes_df.alias("c")
a = atenciones_df.alias("a")

df_completo = (r.join(c, r.id_cliente == c.id_cliente, "left")
                .join(a, r.id_reclamo == a.id_reclamo, "left")
     .select(
         r.id_reclamo,
         r.id_cliente,
         c.nombre.alias("nombre_cliente"),
         c.ciudad,
         r.canal,
         r.tipo_reclamo,
         r.fecha_reclamo,
         a.estado.alias("estado_atencion"),
         a.dias_respuesta
     )
)

df_completo.show(5)



In [ ]:
df_completo.groupBy("tipo_reclamo").agg(
    round(avg("dias_respuesta"), 2).alias("promedio_dias_respuesta")
    ).orderBy("promedio_dias_respuesta").show()

In [ ]:
df_completo.groupBy("tipo_reclamo", "canal").agg(
    count("*").alias("total_reclamos")
    ).orderBy(desc(col("total_reclamos"))).show(6)